In [ ]:
# | default_exp utils/residuals

# Imports

In [ ]:
# | export


from typing import Literal

import torch
from torch import nn

# Setup

In [ ]:
# | export


class Residual(nn.Module):
    """A simple residual connection.

    This has been saved as an nn.Module so that it can always be converted to a stochastic version if required.
    """

    def forward(self, old_value: torch.Tensor, new_value: torch.Tensor):
        """Simply adds the new value to the old value.

        Args:
            old_value: Old value to be added to.
            new_value: New value to be added.

        Returns:
            Value after adding the new value to the old value.
        """
        return old_value + new_value

In [ ]:
# | export


class StochasticDepthResidual(Residual):
    """This class can be wrapped around a list of modules to randomly drop some of them during training."""

    def __init__(self, survival_prob: float = 1.0, dropout_type: Literal["layer", "neuron"] = "layer"):
        """Initializes the StochasticDepthResidual module.

        Use ``dropout_type="layer"`` for most occasions as that implements true stochastic depth dropout as per SOTA
        papers. Use `dropout_type="neuron"` only if you want to drop individual neurons.

        Args:
            survival_prob: Prbability that every layer / neuron will survive the residual connection. Defaults to 1.0.
            dropout_type: Defaults to "layer".
        """

        super().__init__()
        assert 1.0 >= survival_prob > 0.0, "Survival probability must be between (0, 1]"
        self.survival_prob = survival_prob
        self.dropout_type = dropout_type

    def forward(self, old_value: torch.Tensor, new_value: torch.Tensor):
        """Drops the new value with a certain probability and scales the remaining value before adding it to the old
        value. See :py:class:`Residual` for more details.

        Args:
            old_value: Old value to be added to.
            new_value: New value to be added.

        Returns:
            Value after performing stochastic depth and adding the new value to the old value.
        """

        if not self.training:
            return super().forward(old_value, new_value)

        if self.dropout_type == "layer":
            ndim = new_value.ndim
            mask_shape = tuple([new_value.shape[0]] + [1] * (ndim - 1))
        elif self.dropout_type == "neuron":
            mask_shape = new_value.shape

        survival_mask = torch.rand(mask_shape, device=new_value.device) < self.survival_prob
        new_value = (new_value * survival_mask) / self.survival_prob
        return super().forward(old_value, new_value)

    def extra_repr(self) -> str:
        return f"survival_prob={self.survival_prob}, dropout_type='{self.dropout_type}'"

In [ ]:
test = StochasticDepthResidual(0.8)
display(test)

x = torch.randn(10, 3, 32, 32)
test(x, x).shape

StochasticDepthResidual(survival_prob=0.8, dropout_type='layer')

torch.Size([10, 3, 32, 32])

In [ ]:
# | export


def add_stochastic_depth_dropout(module: nn.Module, *args, **kwargs):
    """Converts all instances of :py:class:`Residual` in a module to :py:class:`StochasticDepthResidual`."""

    for name, submodule in module.named_modules():
        if isinstance(submodule, Residual):
            parent_module = module
            for part in name.split(".")[:-1]:
                parent_module = getattr(parent_module, part)
            name = name.split(".")[-1]
            setattr(parent_module, name, StochasticDepthResidual(*args, **kwargs))
    return module

In [ ]:
class A(nn.Module):
    def __init__(self):
        super().__init__()
        self.res1 = Residual()
        self.res2 = Residual()


class B(nn.Module):
    def __init__(self):
        super().__init__()
        self.res = Residual()
        self.a = A()


module = B()
display(module)
module = add_stochastic_depth_dropout(module, 0.75, "neuron")
display(module)


B(
  (res): Residual()
  (a): A(
    (res1): Residual()
    (res2): Residual()
  )
)


B(
  (res): StochasticDepthResidual(survival_prob=0.75, dropout_type='neuron')
  (a): A(
    (res1): StochasticDepthResidual(survival_prob=0.75, dropout_type='neuron')
    (res2): StochasticDepthResidual(survival_prob=0.75, dropout_type='neuron')
  )
)

# nbdev

In [ ]:
!nbdev_export